In [1]:
import numpy as np

In [2]:
from sklearn.metrics import make_scorer

In [3]:
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import category_encoders

In [5]:
import xgboost as xgb

In [6]:
from sklearn.model_selection import GridSearchCV

In [7]:
import pandas as pd

In [8]:
predict = pd.read_csv('trocafone_kaggle_test.csv')

In [10]:
a_predecir = pd.read_csv('labels_training_set.csv')

In [9]:
processed = pd.read_csv('new_processed.csv')

In [11]:
processed = processed.set_index('person')
processed = processed.drop('Unnamed: 0',axis=1)
processed.head()

mayor_evento              prod_visto      dispos  compras  \
person                                                                     
0008ed71           checkout                 Ninguno    Computer        0   
00091926     viewed product               iPhone 6S    Computer        0   
00091a7a      brand listing               iPhone 6S  Smartphone        0   
000ba417     viewed product  Samsung Galaxy A3 Duos    Computer        1   
000c79fe  searched products                iPhone 7  Smartphone        0   

          visitas  cant_prod_vistos        primer_visita        ultima_visita  \
person                                                                          
0008ed71        2                 0  2018-05-17 12:27:47  2018-05-17 16:28:37   
00091926       34               372  2018-05-03 22:08:29  2018-05-31 19:52:03   
00091a7a        1                 3  2018-03-26 14:51:11  2018-03-26 14:56:58   
000ba417        6               153  2018-05-17 11:11:45  2018-05-26 13:09:22   
000c79fe        1                 3  2018-05-29 00:27:47  2018-05-29 00:38:07   

          cant_eventos  permanencia  hace_cuanto_visito  eventos_por_dia  \
person                                                                     
0008ed71             6            1                  14              6.0   
00091926           448           28                   0             16.0   
00091a7a            10            1                  66             10.0   
000ba417           206           10                   5             20.6   
000c79fe            17            1                   2             17.0   

           channel         0         1  
person                                  
0008ed71  Referral -0.186678 -0.103768  
00091926    Direct  0.414402 -0.442355  
00091a7a      Paid  0.414402 -0.442355  
000ba417    Direct -0.303714  0.023156  
000c79fe      Paid  0.668334  0.060729

In [12]:
predict_features = processed.loc[processed.index.isin(predict['person'])].drop(['primer_visita','ultima_visita'],axis=1)

In [13]:
training_features = processed.loc[processed.index.isin(a_predecir['person'])]
training_features['label'] = a_predecir.set_index('person')['label']


training_labels = training_features['label']
training_features = training_features.drop(['primer_visita','ultima_visita','label'],axis=1)

C:\Users\javif\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
training_features.head()

mayor_evento                prod_visto      dispos  compras  \
person                                                                       
0008ed71           checkout                   Ninguno    Computer        0   
000c79fe  searched products                  iPhone 7  Smartphone        0   
001802e4    ad campaign hit                 iPhone 6S  Smartphone        0   
0019e639     viewed product  Motorola Moto G2 3G Dual    Computer        2   
001b0bf9     viewed product    Samsung Galaxy S8 Plus     Ninguno        1   

          visitas  cant_prod_vistos  cant_eventos  permanencia  \
person                                                           
0008ed71        2                 0             6            1   
000c79fe        1                 3            17            1   
001802e4        1                 4            19            1   
0019e639       19               189           471          115   
001b0bf9        0                 2             7            1   

          hace_cuanto_visito  eventos_por_dia   channel         0         1  
person                                                                       
0008ed71                  14         6.000000  Referral -0.186678 -0.103768  
000c79fe                   2        17.000000      Paid  0.668334  0.060729  
001802e4                   0        19.000000      Paid  0.414402 -0.442355  
0019e639                   9         4.095652      Paid -0.195117 -0.188160  
001b0bf9                   9         7.000000   Ninguno -0.358131 -0.144479

In [15]:
tg = category_encoders.TargetEncoder(smoothing=True)
encoded_training = tg.fit_transform(np.array(training_features),np.array(training_labels))
encoded_predict = tg.transform(np.array(predict_features))

In [18]:
encoded_training.head()

0         1         2   3   4    5    6    7   8          9   \
0  0.002778  0.042654  0.068258   0   2    0    6    1  14   6.000000   
1  0.055313  0.050139  0.036301   0   1    3   17    1   2  17.000000   
2  0.031056  0.033590  0.036301   0   1    4   19    1   0  19.000000   
3  0.049121  0.030380  0.068258   2  19  189  471  115   9   4.095652   
4  0.049121  0.060150  0.017361   1   0    2    7    1   9   7.000000   

         10        11        12  
0  0.066079 -0.186678 -0.103768  
1  0.042803  0.668334  0.060729  
2  0.042803  0.414402 -0.442355  
3  0.042803 -0.195117 -0.188160  
4  0.017361 -0.358131 -0.144479

In [19]:
params={
    'n_estimators':[1,2,3,4,5,6,7,8,9,10],
    'max_samples':[1.0],
    'max_features':[1.0], 
}
grid = GridSearchCV(xgb.XGBClassifier(),params,scoring=make_scorer(roc_auc_score),cv=2,verbose=0)

In [20]:
xtr,xte,ytr,yte=train_test_split(encoded_training,training_labels)

In [21]:
grid.fit(xtr,ytr)

C:\Users\javif\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\javif\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\javif\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\javif\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but i

GridSearchCV(cv=2, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'max_samples': [1.0], 'max_features': [1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(roc_auc_score), verbose=0)

In [22]:
grid.score(X=xte,y=yte)

C:\Users\javif\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.5138343480062474

In [23]:
xgbc= grid.best_estimator_

In [24]:
xgbc.fit(xtr,ytr)
p=xgbc.predict(xte)
roc_auc_score(y_score=p,y_true=yte)

C:\Users\javif\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.5138343480062474

In [25]:
xgbc.fit(encoded_training,training_labels)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, max_features=1.0, max_samples=1.0, min_child_weight=1,
       missing=None, n_estimators=1, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [ ]:
prediction_submit = xgbc.predict(encoded_predict)

In [ ]:
predict_features['label'] = prediction_submit

In [ ]:
predict_features['label'].value_counts()

In [ ]:
subm = predict.merge(predict_features['label'].reset_index(),how='left',on='person')

In [ ]:
subm = subm.set_index('person')